In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import keras.utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.regularizers as kr
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import keras.losses as ls
import matplotlib.pyplot as plt
import matplotlib
import sys
import math as m
np.set_printoptions(threshold=sys.maxsize)
%matplotlib qt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

Using TensorFlow backend.


In [2]:
###Vengono caricati i dati dal file, questa volta non viene fatta nessuna selezione a priori
###a parte quella per numero di elementi non nulli.

data=pd.read_csv('./Settings/Data/Default/DEFGBData.csv')
nancount=[]
ind=[]
for i in range(0,data.shape[0]):
    nancount.append(data.iloc[i,12:].isnull().sum())
    if nancount[i]==0:
        ind.append(i)
    else:
        pass
data=data.iloc[ind,:]
data=data.set_index(['GES_FLD'])

data=data.reset_index()   
data1=data.values

dataset=data.values[:,12:].astype(float)
datafei=data.values[:,9].astype(float)
dataset=np.nan_to_num(dataset)
datafei=np.nan_to_num(datafei)
dataset=np.c_[dataset,datafei]

asun=[6.24,7.60,6.47,6.34,3.15,3.15,4.95,3.93,5.64,5.64,7.50,7.50,4.99,6.22,2.21]
for i in range(0,dataset.shape[0]):
    for j in range(0,dataset.shape[1]-1):
        if j==10:
            pass
        else:
            dataset[i,j]=(dataset[i,j]-asun[j])-(dataset[i,10]-asun[10])

dataset=np.delete(dataset,10,1)
invar=dataset

In [3]:
###Selezione del target.
###Target disponibili: 'NGC2243', 'NGC2420', 'Rup134', 'Trumpler20', 'Trumpler23'
target='NGC2243'
path="./Settings/Weights/target"
path=path.replace('target',target)
files=['/gbl0.npy','/gbl0bias.npy','/gbl1.npy','/gbl1bias.npy','/gbl2.npy','/gbl2bias.npy','/gbl3.npy','/gbl3bias.npy']
for i in range(len(files)):
    files[i]=path+files[i]
w0=np.load(files[0])
b0=np.load(files[1])
w1=np.load(files[2])
b1=np.load(files[3])
w2=np.load(files[4])
b2=np.load(files[5])
w3=np.load(files[6])
b3=np.load(files[7])

path2="./Settings/target"
path2=path2.replace('target',target)
files2=['/vallossgb.npy','/trainlossgb.npy','/valaccuracygb.npy','/valsetinputgb.npy','/valsetoutputgb.npy','/valindgb.npy']
for i in range(len(files2)):
    files2[i]=path2+files2[i]
valloss=np.load(files2[0])
loss=np.load(files2[1])
valacc=np.load(files2[2])
valin=np.load(files2[3])
valout=np.load(files2[4])
valind=np.load(files2[5])

In [4]:
###Nuova NN. Vengono caricati i files preparati con il training e associati ai rispettivi layers.

model=Sequential()
model.add(Dense(invar.shape[1],input_dim=invar.shape[1],activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dense(round(invar.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dense(3,activation='softsign'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.layers[0].set_weights([w0,b0])
model.layers[1].set_weights([w1,b1])
model.layers[2].set_weights([w2,b2])

In [5]:
###Viene fatta una run con tutte le stelle, si selezionano quali visualizzare direttamente nei grafici.

out=model.predict(dataset)

In [6]:
dummyout=np_utils.to_categorical(valout)
model2=Sequential()
model2.add(Dense(invar.shape[1],input_dim=invar.shape[1],activation='selu',kernel_regularizer=kr.l1(0.01)))
model2.add(Dense(round(invar.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.01)))
model2.add(Dense(3,activation='softsign'))
model2.add(Dense(dummyout.shape[1],activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model2._estimator_type='classifier'
model2.layers[0].set_weights([w0,b0])
model2.layers[1].set_weights([w1,b1])
model2.layers[2].set_weights([w2,b2])
model2.layers[3].set_weights([w3,b3])
model2.classes_ = 0,1
model2.dummy_='dummy'

out2=model2.predict(valin)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dummyout.argmax(axis=1),out2.argmax(axis=1))
prec=cm[1,1]/(cm[1,1]+cm[0,1])
recall=cm[1,1]/(cm[1,1]+cm[1,0])
accur=(cm[1,1]+cm[0,0])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
fscore=cm[1,1]/(cm[1,1]+0.5*(cm[1,0]+cm[0,1]))
print('Target: ',target)
print('TP = ',cm[1,1],', TN = ',cm[0,0],', FP = ',cm[0,1],', FN = ',cm[1,0])
print('Precision = ',prec,', Recall = ',recall,', Accuracy = ',accur)
print('F-score = ',fscore)


Target:  NGC2243
TP =  11 , TN =  74 , FP =  6 , FN =  2
Precision =  0.6470588235294118 , Recall =  0.8461538461538461 , Accuracy =  0.9139784946236559
F-score =  0.7333333333333333


In [8]:
out3=model2.predict(dataset)

In [8]:
from sklearn.inspection import plot_partial_dependence
elem = ['[Na1/Fe]','[Mg1/Fe]','[Al1/Fe]','[Ca1/Fe]','[Sc1/Fe]','[Sc2/Fe]','[Ti1/Fe]','[Va1/Fe]','[Cr1/Fe]','[Cr2/Fe]','[Fe2/Fe]','[Co1/Fe]','[Ni1/Fe]','[Y1/Fe]','[Fe/H]']

In [9]:
fig, ax = plt.subplots(figsize=(17, 15))
ax.set_title(target,fontsize=28)
mlp_disp = plot_partial_dependence(model2,dataset,target=1,features=[0,1,2,3,4,5],feature_names=elem,kind='both',ax=ax,percentiles=(0.00,1.00))

In [10]:
fig, ax = plt.subplots(figsize=(17, 15))
ax.set_title(target,fontsize=28)
mlp_disp = plot_partial_dependence(model2,dataset,target=1,features=[6,7,8,9,10,11],feature_names=elem,kind='both',ax=ax,percentiles=(0.00,1.00))

In [13]:
fig, ax = plt.subplots(figsize=(17, 5))
ax.set_title(target,fontsize=28)
mlp_disp = plot_partial_dependence(model2,dataset,target=1,features=[12,13,14],feature_names=elem,kind='both',ax=ax,percentiles=(0.00,1.00))

In [14]:
import time
start = time.time()

indel=[2,4,6,8,11,13,14]
for i in range(len(indel)):
    step = time.time()
    fig, ax = plt.subplots(figsize=(17, 15))
    feat=[]
    for j in range(len(indel)):
        if j==i:
            pass
        else:
            feat.append((indel[i],indel[j]))
    title='target, element'
    title=title.replace('target',target)
    title=title.replace('element',elem[indel[i]])
    ax.set_title(title,fontsize=28)
    mlp_disp = plot_partial_dependence(model2,dataset,features=feat,feature_names=elem,kind='average',ax=ax,percentiles=(0.00,1.00))
    print('Step ',i,':',"--- %s seconds ---" % (time.time() - step))

print("--- %s seconds ---" % (time.time() - start))

Step  0 : --- 153.6030571460724 seconds ---
Step  1 : --- 149.85823798179626 seconds ---
Step  2 : --- 120.01253199577332 seconds ---
Step  3 : --- 85.15466594696045 seconds ---
Step  4 : --- 133.19202065467834 seconds ---
Step  5 : --- 153.62458324432373 seconds ---
Step  6 : --- 159.17383980751038 seconds ---
--- 954.6199371814728 seconds ---
